[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/Simpy/4_Processo_aula_paralelo.ipynb)

# Modelagem e Gestão de Processos


**Prof. Diogo Ferreira de Lima Silva (TEP-UFF)**



In [ ]:
!pip install simpy

In [2]:
import simpy
import random
import numpy as np

In [11]:
random.seed(10)

### Analisando o que seria esperado

<img src="processo_aula3.png">


Imagine que não temos fila os tempos da figura são os tempos de processameto reais (determinísticos) das atividades.

Temos:

- Tempo de Ciclo no pior caso: o trabalho passa por B e também precisa de retrabalho

$$ TC_{pior} = 10 + 20 + 23 + 2 \times 35 + \max{(13;9)} + 2 = 138 $$

- Tempo de Ciclo no melhor caso: o trabalho não passa por B e não precisa de retrabalho

$$ TC_{melhor} = 10 + 23 + 35 + \max{(13;9)} + 2 = 83 $$

- Tempo de Ciclo no esperado (retrabalho só pode ser feito 1 vez):

$$TC_{esperado} = 10 + 0,3 \times 20 + 23 + 1,1 \times (10+20+5)+ \max{(13;9)} +2 = 92,5$$

- Tempo de Ciclo no esperado (retrabalho pode ser feito mais de uma vez):

$$TC_{esperado} = 10 + 0,3 \times 20 + 23 + \frac{10+20+5}{1-0,1} + \max{(13;9)} +2 = 92,89$$

### Reescrevendo o código

Para deixar nosso código mais organizado, vamos criar uma função para cada atividade.

In [12]:
def atividade_A (environment, trabalho_id):
    with R1.request() as req:
        yield req
        yield environment.timeout(10)
        
def atividade_B (environment, trabalho_id):
    with R1.request() as req:
        yield req
        yield environment.timeout(20)  
        
def atividade_C (environment, trabalho_id):
    with R2.request() as req:
        yield req
        yield environment.timeout(23)  
        
def atividade_D (environment, trabalho_id):
    with R2.request() as req:
        yield req
        yield environment.timeout(10)  

def atividade_E (environment, trabalho_id):
    with R2.request() as req:
        yield req
        yield environment.timeout(20)  
        
def atividade_F (environment, trabalho_id):
    with R1.request() as req:
        yield req
        yield environment.timeout(9)  
        
def atividade_G (environment, trabalho_id):
    with R3.request() as req:
        yield req
        yield environment.timeout(13)  
        
def atividade_H (environment, trabalho_id):
    with R3.request() as req:
        yield req
        yield environment.timeout(2)  
        
def atividade_I (environment, trabalho_id):
    yield environment.timeout(5)  
    


Vamos modelar o processo completo. Veja que agora o código fica bem mais limpo e organizado.

Usaremos um input adicional para determinar se uma tarefa pode entrar no ciclo de retrabalho mais de uma vez.

In [13]:
def processo_aula3 (environment, trabalho_id, R1, R2, R3, retrabalho_repete):
        #print(f'tempo: {environment.now} -- {trabalho_id} entrou no processo')
        entrou_no_processo = environment.now

        # Atividade A
        yield (environment.process(atividade_A(environment, trabalho_id)))
   
        # Atividade B só acontece em 30% dos trabalhos
        regra_B = random.uniform(0,1) 
        if regra_B < 0.3:
                yield (environment.process(atividade_B(environment, trabalho_id)))
    
        # Atividade C
        yield(environment.process(atividade_C(environment, trabalho_id)))

        # Atividade D
        yield(environment.process(atividade_D(environment, trabalho_id)))
        
        # Atividade E
        yield(environment.process(atividade_E(environment, trabalho_id)))

        # Atividade I
        yield (environment.process(atividade_I(environment, trabalho_id)))
        
        #Regra de Retrabalho
        regra_I = random.uniform(0,1)
        
        if retrabalho_repete == True:
                while regra_I < 0.1:
                        # Atividade D
                        yield(environment.process(atividade_D(environment, trabalho_id)))
                        # Atividade E
                        yield(environment.process(atividade_E(environment, trabalho_id)))
                        # Atividade I
                        yield (environment.process(atividade_I(environment, trabalho_id)))
                        regra_I = random.uniform(0,1)
        else:
                if regra_I < 0.1:
                        # Atividade D
                        yield(environment.process(atividade_D(environment, trabalho_id)))
                        # Atividade E
                        yield(environment.process(atividade_E(environment, trabalho_id)))
                        # Atividade I
                        yield (environment.process(atividade_I(environment, trabalho_id)))
                        regra_I = random.uniform(0,1)


        # Atividades F e G são paralelas!
        a_lista = []
        for a in superatividade:
                a_lista.append(environment.process(a(environment, trabalho_id)))
        yield environment.all_of(a_lista)     
        
        # Atividade H
        yield(environment.process(atividade_H(environment, trabalho_id)))
        
        #print(f'tempo: {environment.now} -- {trabalho_id} saiu do processo')
        
        saiu_do_processo = environment.now
        tempo_de_ciclo.append (saiu_do_processo - entrou_no_processo) 


In [14]:
def chegadas (environment, retrabalho_repete):
    id = 1 #guarda o id do cliente 
    # Enquanto houver simulação:
    while True:
        # Passa um tempo até a próxima chegada
        yield environment.timeout(140)
        
        # Um cliente chega no processo
        environment.process(processo_aula3 (environment, 'Trabalho %d' % id, R1, R2, R3, retrabalho_repete))
        
        # O próximo cliente terá id = id + 1
        id += 1

#### Condiderando 1 retrabalho no máximo por entrada

In [15]:
TC=[]

# Vamos rodar a simulação 30 vezes e guardar o Tempo de Ciclo Médio de cada uma!
for i in range (30):
    espera_A, espera_B, espera_C, espera_D, espera_E, espera_F,espera_G, espera_H, espera_I = [],[],[],[],[],[],[],[],[]
    processamento_A, processamento_B, processamento_C, processamento_D, processamento_E, processamento_F,processamento_G, processamento_H, processamento_I = [],[],[],[],[],[],[],[],[] 
    tempo_de_ciclo = []
    ambiente = simpy.Environment()
    R1 = simpy.Resource(ambiente, capacity=1)
    R2 = simpy.Resource(ambiente, capacity=1)
    R3 = simpy.Resource(ambiente, capacity=1)
    superatividade = [atividade_F, atividade_G]
    ambiente.process(chegadas (ambiente, retrabalho_repete = False))
    ambiente.run(until = 300000)

    TC.append(np.mean(tempo_de_ciclo))


In [16]:
print (f"A média dos tempos de ciclo médios das 30 simulações foi: {np.mean(TC)}")
print (f"Na última simulação, o tempo de ciclo máximo foi {np.max(tempo_de_ciclo)}")
print (f"Na última simulação, o tempo de ciclo mínimo foi {np.min(tempo_de_ciclo)}")


A média dos tempos de ciclo médios das 30 simulações foi: 92.49735300731949
Na última simulação, o tempo de ciclo máximo foi 138
Na última simulação, o tempo de ciclo mínimo foi 83


#### Possibilitando vários retrabalhos

In [17]:
TC=[]

# Vamos rodar a simulação 30 vezes e guardar o Tempo de Ciclo Médio de cada uma!
for i in range (30):
    espera_A, espera_B, espera_C, espera_D, espera_E, espera_F,espera_G, espera_H, espera_I = [],[],[],[],[],[],[],[],[]
    processamento_A, processamento_B, processamento_C, processamento_D, processamento_E, processamento_F,processamento_G, processamento_H, processamento_I = [],[],[],[],[],[],[],[],[] 
    tempo_de_ciclo = []
    ambiente_2 = simpy.Environment()
    R1 = simpy.Resource(ambiente_2, capacity=1)
    R2 = simpy.Resource(ambiente_2, capacity=1)
    R3 = simpy.Resource(ambiente_2, capacity=1)
    superatividade = [atividade_F, atividade_G]
    ambiente_2.process(chegadas (ambiente_2, retrabalho_repete = True)) # A MUDANÇA FOI AQUI!
    ambiente_2.run(until = 300000)
    TC.append(np.mean(tempo_de_ciclo))


In [18]:
print (f"A média dos tempos de ciclo médios das 30 simulações foi: {np.mean(TC)}")
print (f"Na última simulação, o tempo de ciclo máximo foi {np.max(tempo_de_ciclo)}")
print (f"Na última simulação, o tempo de ciclo mínimo foi {np.min(tempo_de_ciclo)}")

A média dos tempos de ciclo médios das 30 simulações foi: 92.98781519873505
Na última simulação, o tempo de ciclo máximo foi 286
Na última simulação, o tempo de ciclo mínimo foi 83


Interessante! Agora tivemos um tempo de ciclo máximo mais alto do que o esperado...

Isso ocorre porque possivelmente houve espera por recursos no retrabalho!

Também pode acontecer se um mesmo trabalho passou várias vezes por retrabalho.

---

### Exercício

Faça os seguintes testes buscando que o máximo tempo de ciclo encontrado diminua.

- Aumente o suficiente a capacidade dos recurso. 
- Aumente o tempo entre chegadas.